In [5]:
import numpy as np
import pandas as pd
import xarray as xr

import os
from glob import glob

import matplotlib.pyplot as plt

import histlib.matchup as match
import histlib.diagnosis as diag
from histlib.cstes import labels, zarr_dir, matchup_dir, var
from histlib.matchup import _data_var, _stress_var, _aviso_var

from xhistogram.xarray import histogram


import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo
crs = ccrs.PlateCarree()
import cmocean.cm as cm


In [6]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    #cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=28, processes=28, walltime='02:00:00')
    w = cluster.scale(jobs=1)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.62:8787/status,
Dashboard: http://10.148.1.62:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.62:58839,Workers: 0
Dashboard: http://10.148.1.62:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## IMPORT DATA, build dataset
We choose to get only erastar wind term, and we reject PEACHY data, and we select only deltaT<1H

# For one label

In [7]:
l = "gps_Sentinel-3_A_2019"

In [8]:
dl = 5e3
dt = 10*60

DL =25e3 #meters
DT = 0.5*3600 #seconds

#IF restricted computation
wd_x= ['es_cstrio_z15_alti_wd_x', 'es_cstrio_z15_drifter_wd_x']
wd_y = []
grad_x = ['alti_ggx_adt_filtered',
          'alti_ggx_adt_filtered_ocean_tide',
          'alti_ggx_adt_filtered_ocean_tide_internal_tide',
          'alti_ggx_adt_filtered_ocean_tide_internal_tide_dac',
         'aviso_alti_ggx_adt',
          'aviso_drifter_ggx_adt'
         ]
grad_y =[]
cutoff=[0, 2.5]

var = wd_x+grad_x+grad_y+['drifter_acc_x_0','drifter_coriolis_x_0']

def bin_dist_ms(ds,l, dl):
    dsm = match.add_except_sum(ds, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff)  
    dsm = dsm.reset_coords(['lon', 'lat', 'time']).drop(['id_comb', 'time'])
    dfm = dsm.to_dask_dataframe().set_index('obs')
    dfm["distbin"] = (dfm.alti___distance // dl) * dl
    #ms
    d2 = dfm.drop(['lon', 'lat', 'distbin'], axis=1)**2
    dd = dfm[['distbin']].merge(d2)
    dd = dd.groupby(["distbin"]).mean()
    #count
    dnb = dfm.reset_index()[['obs', 'distbin']].groupby(["distbin"]).count().obs.compute().to_xarray()
    dsms = dd.compute().to_xarray()
    #attrs
    for v in list(dsms.variables) :
        if v in ds :
            dsms[v].attrs = ds[v].attrs
    #merge
    dso = xr.merge([dsms, dnb.rename('nb_coloc_bin')])
    dso['drifter_sat_year']=l
    dso = dso.expand_dims('drifter_sat_year')
    dso = dso.set_coords('drifter_sat_year')
    # center lon, lat bins + reindex to have same for all
    dist_bins = np.arange(0, 200e3, dl)
    dso = dso.reindex({'distbin':dist_bins})
    dso['distbin'] = dso['distbin']+dl/2

    return dso

def bin_dist_mean(ds,l, dl):
    dsm = match.add_except_sum(ds, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff)      
    dsm = dsm.reset_coords(['lon', 'lat', 'time']).drop(['id_comb', 'time'])
    dfm = dsm.to_dask_dataframe().set_index('obs')
    dfm["distbin"] = (dfm.alti___distance // dl) * dl
    
    #mean
    d2 = dfm.drop(['lon', 'lat', 'distbin'], axis=1)
    dd = dfm[['distbin']].merge(d2)
    dd = dd.groupby(["distbin"]).mean()
    
    #count
    dnb = dfm.reset_index()[['obs', 'distbin']].groupby(["distbin"]).count().obs.compute().to_xarray()
    dsms = dd.compute().to_xarray()
    #attrs
    for v in list(dsms.variables) :
        if v in ds :
            dsms[v].attrs = ds[v].attrs
    #merge
    dso = xr.merge([dsms, dnb.rename('nb_coloc_bin')])
    dso['drifter_sat_year']=l
    dso = dso.expand_dims('drifter_sat_year')
    dso = dso.set_coords('drifter_sat_year')
    # center lon, lat bins + reindex to have same for all
    dist_bins = np.arange(0, 200e3, dl)
    dso = dso.reindex({'distbin':dist_bins})
    dso['distbin'] = dso['distbin']+dl/2

    return dso


def bin_time_ms(ds,l, dt):
    dsm = match.add_except_sum(ds, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff)   
    dsm = dsm.reset_coords(['lon', 'lat', 'time']).drop(['id_comb', 'time'])
    dfm = dsm.to_dask_dataframe().set_index('obs')
    dfm["timebin"] = (dfm.alti___time_difference // dt) * dt
    #ms
    d2 = dfm.drop(['lon', 'lat', 'timebin'], axis=1)**2
    dd = dfm[['timebin']].merge(d2)
    dd = dd.groupby(["timebin"]).mean()
    #count
    dnb = dfm.reset_index()[['obs', 'timebin']].groupby(["timebin"]).count().obs.compute().to_xarray()
    dsms = dd.compute().to_xarray()
    #attrs
    for v in list(dsms.variables) :
        if v in ds :
            dsms[v].attrs = ds[v].attrs
    #merge
    dso = xr.merge([dsms, dnb.rename('nb_coloc_bin')])
    dso['drifter_sat_year']=l
    dso = dso.expand_dims('drifter_sat_year')
    dso = dso.set_coords('drifter_sat_year')
    # center lon, lat bins + reindex to have same for all
    time_bins = np.arange(0, 3*3600, dt)
    dso = dso.reindex({'timebin':time_bins})
    dso['timebin'] = dso['timebin']+dt/2

    return dso

def bin_time_mean(ds,l, dt):
    dsm = match.add_except_sum(ds, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff)   
    dsm = dsm.reset_coords(['lon', 'lat', 'time']).drop(['id_comb', 'time'])
    dfm = dsm.to_dask_dataframe().set_index('obs')
    dfm["timebin"] = (dfm.alti___time_difference // dt) * dt
    #ms
    d2 = dfm.drop(['lon', 'lat', 'timebin'], axis=1)
    dd = dfm[['timebin']].merge(d2)
    dd = dd.groupby(["timebin"]).mean()
    #count
    dnb = dfm.reset_index()[['obs', 'timebin']].groupby(["timebin"]).count().obs.compute().to_xarray()
    dsms = dd.compute().to_xarray()
    #attrs
    for v in list(dsms.variables) :
        if v in ds :
            dsms[v].attrs = ds[v].attrs
    #merge
    dso = xr.merge([dsms, dnb.rename('nb_coloc_bin')])
    dso['drifter_sat_year']=l
    dso = dso.expand_dims('drifter_sat_year')
    dso = dso.set_coords('drifter_sat_year')
    # center lon, lat bins + reindex to have same for all
    time_bins = np.arange(0, 3*3600, dt)
    dso = dso.reindex({'timebin':time_bins})
    dso['timebin'] = dso['timebin']+dt/2

    return dso

## Dist

In [9]:
dsm = xr.open_dataset(os.path.join(matchup_dir, f'matchup_{l}.zarr'))[var+['drogue_status', 'alti___distance', 'alti___time_difference']].dropna('obs').chunk({'obs':500})
# add filtered acc and coriolis
zarr_cutoff = os.path.join(zarr_dir+'_ok','cutoff_matchup',"cutoff_matchup_"+l+".zarr")
zarr_cutoff1 = os.path.join(zarr_dir+'_ok','cutoff_matchup',"cutoff_matchup_"+l+"_2.zarr")
if os.path.isdir(zarr_cutoff):
    dsm = xr.merge([dsm, xr.open_dataset(zarr_cutoff),xr.open_dataset(zarr_cutoff1)], join='inner').chunk({'obs':1000})
    
dsm = dsm.where(dsm.alti___time_difference<=DT, drop=True).drop(['alti___time_difference']).dropna('obs')
dsmd = dsm.where(dsm.drogue_status, drop=True).drop('drogue_status')
dsmnd = dsm.where(np.logical_not(dsm.drogue_status), drop=True).drop('drogue_status')
dsm = dsm.drop('drogue_status')

In [10]:
dsm

<xarray.Dataset>
Dimensions:                                             (obs: 286385)
Coordinates:
  * obs                                                 (obs) int64 0 ... 325270
    alti___distance                                     (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    lat                                                 (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    lon                                                 (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    time                                                (obs) datetime64[ns] dask.array<chunksize=(999,), meta=np.ndarray>
Data variables: (12/24)
    es_cstrio_z15_alti_wd_x                             (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    es_cstrio_z15_drifter_wd_x                          (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    alti_ggx_adt_filtered                               (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    alti_ggx_adt_filtered_ocean_tide                    (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    alti_ggx_adt_filtered_ocean_tide_internal_tide      (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    alti_ggx_adt_filtered_ocean_tide_internal_tide_dac  (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    ...                                                  ...
    drifter_coriolis_x_1                                (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    drifter_coriolis_x_2                                (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    drifter_acc_x_15                                    (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    drifter_acc_x_25                                    (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    drifter_coriolis_x_15                               (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
    drifter_coriolis_x_25                               (obs) float64 dask.array<chunksize=(999,), meta=np.ndarray>
Attributes:
    __id:                   CMEMS-L3-SLA-Sentinel-3_A__gps
    __time_coverage_end:    2020-01-01T00:00:00
    __time_coverage_start:  2019-01-01T00:00:00

In [11]:
zarr = os.path.join(zarr_dir+'_ok',f'msdist/msdist_{int(dl/1e3)}_{l}.zarr')
bin_dist_ms(dsm,l, dl).to_zarr(zarr,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')

In [12]:
zarrd = os.path.join(zarr_dir+'_ok',f'msdist/msdist_{int(dl/1e3)}_drogued_{l}.zarr')
if not os.path.isdir(zarrd) :
    bin_dist_ms(dsmd,l, dl).to_zarr(zarrd, encoding={'drifter_sat_year':{'dtype':'U32'}},mode='w')
zarrud = os.path.join(zarr_dir+'_ok',f'msdist/msdist_{int(dl/1e3)}_undrogued_{l}.zarr')
if not os.path.isdir(zarrud) :
    bin_dist_ms(dsmnd,l, dl).to_zarr(zarrud,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')

In [13]:
zarr = os.path.join(zarr_dir+'_ok',f'meandist/meandist_{int(dl/1e3)}_{l}.zarr')
if not os.path.isdir(zarr) :
    bin_dist_mean(dsm,l, dl).to_zarr(zarr,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
if dsmd.dims['obs']!=0 : 
    zarrd = os.path.join(zarr_dir+'_ok',f'meandist/meandist_{int(dl/1e3)}_drogued_{l}.zarr')
    if not os.path.isdir(zarrd) :
        bin_dist_mean(dsmd,l, dl).to_zarr(zarrd, encoding={'drifter_sat_year':{'dtype':'U32'}},mode='w')
if dsmnd.dims['obs']!=0 :
    zarrud = os.path.join(zarr_dir+'_ok',f'meandist/meandist_{int(dl/1e3)}_undrogued_{l}.zarr')
    if not os.path.isdir(zarrud) :
        bin_dist_mean(dsmnd,l, dl).to_zarr(zarrud,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')



## Time

In [14]:
dt = 10*60

In [15]:
dsm = xr.open_dataset(os.path.join(matchup_dir, f'matchup_{l}.zarr'))[var+['drogue_status', 'alti___distance', 'alti___time_difference']].dropna('obs').chunk({'obs':500})
# add filtered acc and coriolis
zarr_cutoff = os.path.join(zarr_dir+'_ok','cutoff_matchup',"cutoff_matchup_"+l+".zarr")
zarr_cutoff1 = os.path.join(zarr_dir+'_ok','cutoff_matchup',"cutoff_matchup_"+l+"_2.zarr")
if os.path.isdir(zarr_cutoff):
    dsm = xr.merge([dsm, xr.open_dataset(zarr_cutoff),xr.open_dataset(zarr_cutoff1)], join='inner').chunk({'obs':1000})
    
dsm = dsm.where(dsm.alti___distance<=DL, drop=True).drop(['alti___distance']).dropna('obs')
dsmd = dsm.where(dsm.drogue_status, drop=True).drop('drogue_status')
dsmnd = dsm.where(np.logical_not(dsm.drogue_status), drop=True).drop('drogue_status')
dsm = dsm.drop('drogue_status')

In [16]:
# TIME MS
zarr = os.path.join(zarr_dir+'_ok',f'mstime/mstime_{int(dt)}_{l}.zarr')
bin_time_ms(dsm,l, dt).to_zarr(zarr,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')

if dsmd.dims['obs']!=0 : 
    zarrd = os.path.join(zarr_dir+'_ok',f'mstime/mstime_{int(dt)}_drogued_{l}.zarr')
    bin_time_ms(dsmd,l, dt).to_zarr(zarrd, encoding={'drifter_sat_year':{'dtype':'U32'}},mode='w')

if dsmnd.dims['obs']!=0 :
    zarrud = os.path.join(zarr_dir+'_ok',f'mstime/mstime_{int(dt)}_undrogued_{l}.nc')
    bin_time_ms(dsmnd,l, dt).to_zarr(zarrud,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')


# time MEAN
zarr = os.path.join(zarr_dir+'_ok',f'meantime/meantime_{int(dt)}_{l}.zarr')
bin_time_mean(dsm,l, dt).to_zarr(zarr,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')

if dsmd.dims['obs']!=0 : 
    zarrd = os.path.join(zarr_dir+'_ok',f'meantime/meantime_{int(dt)}_drogued_{l}.zarr')
    bin_time_mean(dsmd,l, dt).to_zarr(zarrd, encoding={'drifter_sat_year':{'dtype':'U32'}},mode='w')

if dsmnd.dims['obs']!=0 :
    zarrud = os.path.join(zarr_dir+'_ok',f'meantime/meantime_{int(dt)}_undrogued_{l}.nc')
    bin_time_mean(dsmnd,l, dt).to_zarr(zarrud,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')


____________

In [17]:
cluster.close()